Autorius: *Rugilė Vasaitytė* *s2110643*

Variantas: *modelis resnet50; klasės ["Scorpion", "Goldfish", "Teapot"]*

In [1]:
import os
import numpy as np
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
from torchvision.datasets import ImageFolder
import numpy as np

In [ ]:
%pip install openimages
from openimages.download import download_dataset

In [3]:
def download_openimages(data_dir, classes, number_for_samples):
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    download_dataset(data_dir, classes, limit=number_for_samples)

In [4]:
def create_data_loader(data_dir, batch_size=32, num_workers=4, prefetch_factor=2):
    # Define transformations
    transform = transforms.Compose([
        transforms.Resize((224, 224)),   # Resize images to 224x224
        transforms.ToTensor(),           # Convert images to PyTorch tensors
        transforms.Normalize(            # Normalize images
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])

    # Create dataset
    dataset = ImageFolder(root=data_dir, transform=transform)

    # Create DataLoader
    data_loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        prefetch_factor=prefetch_factor,
        persistent_workers=True
    )

    return data_loader

In [5]:
def calculate_info_in_respect_to_positive_class(prediction_matrix, class_id, true_labels, class_label):
    result = []
    for i, label in enumerate(true_labels):
        match_flag = 1.0 if class_label == label else 0.0 # flags the sample as TRUE when it actually belongs to a POSITIVE class
        result.append((prediction_matrix[i][class_id], match_flag)) # tuple => (prediction probability, does it actually belong to the POSITIVE class)
    return result

In [12]:
def calculate_stats(label_match, predictions, threshold):
  is_prediction_above_threshold = (predictions > threshold).astype(np.float64)
  # True Positives: Predicted positive and actually positive
  TP = np.sum((label_match == 1) & (is_prediction_above_threshold == 1))
  # True Negatives: Predicted negative and actually negative
  TN = np.sum((label_match == 0) & (is_prediction_above_threshold == 0))
  # False Positives: Predicted positive but actually negative
  FP = np.sum((label_match == 0) & (is_prediction_above_threshold == 1))
  # False Negatives: Predicted negative but actually positive
  FN = np.sum((label_match == 1) & (is_prediction_above_threshold == 0))
  stats = {}

  denominator = TP + FP + TN + FN # for avoiding dividing by zero
  stats['Accuracy'] = (TP + TN) / denominator if denominator != 0 else 0.0

  denominator_precision = TP + FP
  stats['Precision'] = TP / denominator_precision if denominator_precision != 0 else 0.0

  denominator_recall = TP + FN
  stats['Recall'] = TP / denominator_recall if denominator_recall != 0 else 0.0

  denominator_f1 = stats['Precision'] + stats['Recall']
  stats['f1'] = 2 * (stats['Precision'] * stats['Recall']) / denominator_f1 if denominator_f1 != 0 else 0.0

  return stats

In [7]:
THRESHOLD = [0.71, 0.8, 0.98]

In [8]:
# Main #
data_dir = "data"
classes = ["Scorpion", "Goldfish", "Teapot"] # true labels 0,1,2
class_idx = [71, 1, 849] # to find the predicted probability for the specified class in the probability vector
number_for_samples = 334
#-------------
download_openimages(data_dir, classes, number_for_samples)

data_loader = create_data_loader(data_dir)

model = models.resnet50(pretrained=True)
model.eval()

# used for storing corresponding values(tuples):
# sample's modeled probability to belong to a positive class and whether it actually had a ground label of the positive class
class_info = [[] for _ in range(len(classes))]

100%|██████████| 41/41 [00:01<00:00, 25.90it/s]
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 91.1MB/s]


In [9]:
for images, labels in data_loader:
  predictions = torch.sigmoid(model(images)).detach().numpy()
  ground_labels = labels.numpy()

  for i, class_id in enumerate(class_idx):
            class_info[i].extend(
                calculate_info_in_respect_to_positive_class(predictions, class_id, ground_labels, i))

In [13]:
# Print statistics for each image class
for i, class_name in enumerate(classes):
        tuple_ = class_info[i]
        pred = [prob for prob, _ in tuple_]
        matches = [match_l for _, match_l in tuple_] # ground truth matches or positive/negative classification of classess
        print(f"{class_name} class: {calculate_stats(np.array(matches), np.array(pred), THRESHOLD[i])}")

Scorpion class: {'Accuracy': 0.5903614457831325, 'Precision': 0.5906432748538012, 'Recall': 0.6047904191616766, 'f1': 0.5976331360946745}
Goldfish class: {'Accuracy': 0.2756024096385542, 'Precision': 0.08274231678486997, 'Recall': 0.2734375, 'f1': 0.12704174228675136}
Teapot class: {'Accuracy': 0.9186746987951807, 'Precision': 0.8245614035087719, 'Recall': 0.9306930693069307, 'f1': 0.8744186046511628}
